# Self-supervised learning with fastai (inference)

This is an inference notebook. You can find the training notebook **[here](https://www.kaggle.com/ankursingh12/shopee-swav-training)**

In this notebook, we will see how our fine-tuned Resnet50 performs against the following:
- Pretrained Resnet50 (from pytorch)
- Resnet50 trained using SwAV (by facebook)

Hold tight, its time for the final verdict (just kidding) . . .

In [13]:
!pip install self-supervised -q

In [14]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

In [15]:
import gc
import os 
import cv2
import timm
import random

import numpy as np 
import pandas as pd 
from tqdm import tqdm
from pathlib import Path

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

from self_supervised.augmentations import *
from self_supervised.layers import *
from self_supervised.vision.swav import *

import torchvision.models as models
from cuml.neighbors import NearestNeighbors

import albumentations as A 
from albumentations.pytorch.transforms import ToTensorV2

In [16]:
path = Path('../input/shopee-product-matching')

class CFG:
    img_size = 512
    batch_size = 12
    seed = 2020
    
    device = 'cuda'
    classes = 11014
    
    scale = 30 
    margin = 0.5

In [17]:
def seed_torch(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(CFG.seed)

### Helper Functions

In [18]:
def read_dataset():
    df = pd.read_csv(path/'train.csv')
    tmp = df.groupby('label_group').posting_id.agg('unique').to_dict()
    df['target'] = df.label_group.map(tmp)
    
    image_paths = str(path) + '/train_images/' + df['image']
    return df, image_paths

def get_test_transforms():
    return A.Compose([A.Resize(CFG.img_size, CFG.img_size, always_apply=True),
                      A.Normalize(), ToTensorV2(p=1.0)])

class ShopeeDataset(Dataset):
    def __init__(self, image_paths, transforms=None):
        self.image_paths = image_paths
        self.augmentations = transforms

    def __len__(self):
        return self.image_paths.shape[0]

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']       
    
        return image,torch.tensor(1)

In [19]:
def get_image_embeddings(image_paths, model, model_path=None):
    embeds = []
    model.eval()
    
    if model_path:
        model.load_state_dict(torch.load(model_path))
        model = model.to(CFG.device)
    
    image_dataset = ShopeeDataset(image_paths,transforms = get_test_transforms())
    image_loader = DataLoader(image_dataset, batch_size=CFG.batch_size, pin_memory=True, 
                              drop_last=False,num_workers=4)
    
    with torch.no_grad():
        for img,label in tqdm(image_loader): 
            img = img.cuda()
            feat = model(img)
            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
            
    image_embeddings = np.concatenate(embeds)
    print(f'Our image embeddings shape is {image_embeddings.shape}')
    
    del embeds, model
    gc.collect()
    return image_embeddings


def get_image_predictions(df, embeddings,threshold = 0.0):
    if len(df) > 3: KNN = 50
    else : KNN = 3
    
    model = NearestNeighbors(n_neighbors = KNN, metric = 'euclidean')
    model.fit(embeddings)
    distances, indices = model.kneighbors(embeddings)
    
    predictions = []
    for k in tqdm(range(embeddings.shape[0])):
        idx = np.where(distances[k,] < threshold)[0]
        ids = indices[k,idx]
        posting_ids = df['posting_id'].iloc[ids].values
        predictions.append(posting_ids)
        
    del model, distances, indices
    gc.collect()
    return predictions

In [20]:
def combine_predictions(row):
    x = np.concatenate([row['image_predictions']])
    return ' '.join( np.unique(x))

def getMetric(row, col):
        n = len(np.intersect1d(row.target,row[col]))
        return 2*n / (len(row.target)+len(row[col]))
    
def evaluate_model(models):
    img_embeddings = 0
    
    if isinstance(models, list):
        for m in models:
            img_embeddings += get_image_embeddings(image_paths.values, m)
        img_embeddings /= len(models)
    else:
        img_embeddings = get_image_embeddings(image_paths.values, models)
        
    img_embeddings = img_embeddings.squeeze()
    image_predictions = get_image_predictions(df, img_embeddings, threshold = 0.36)
    
    df['image_predictions'] = image_predictions
    f1_scores = df.apply(lambda r: getMetric(r, 'image_predictions'), axis=1)
    print(f'CV score for baseline = {f1_scores.mean()}')

### Reading data

In [21]:
df,image_paths = read_dataset()
df.head()

# Inference 

Lets evalutate how our model performs

### 1. Pretrained Resnet50 from pytorch

In [22]:
## Pretrained Resnet50 from pytorch
model = models.resnet50(pretrained=True)
model_enc = torch.nn.Sequential(*(list(model.children())[:-1])).cuda()
model_enc.eval()
evaluate_model(model_enc)
# euclidean -  CV score = 0.4930072045298856

### 2. Resnet50 trained using SwAV, by faceboook

In [23]:
# ## Resnet50 trained using SwAV, by faceboook
# model = torch.hub.load('facebookresearch/swav', 'resnet50')
# model_enc = torch.nn.Sequential(*(list(model.children())[:-1])).cuda()
# model_enc.eval()
# evaluate_model(model_enc)
# # euclidean - CV score = 0.5134202771430936

Nice, from 0.49 to 0.51 just by using weights from facebook.

### 3. Fine-tuned Resnet50 

Finally, lets test our model

In [24]:
## Fine-tuned Resnet50
arch = 'resnet50'
encoder_path = '../input/shopee-swav-training/models/swav_iwang_sz224_epc5_encoder.pth'
encoder = create_encoder(arch, pretrained=False, n_in=3)
encoder.load_state_dict(torch.load(encoder_path))
encoder.cuda()
encoder.eval()
evaluate_model(encoder)

### Important resources 

- Want to train your own model using SwAV, refer [training notebook](https://www.kaggle.com/ankursingh12/shopee-swav-training)
- Want to learn more about SwAV, refer [this](https://www.kaggle.com/ayuraj/v2-self-supervised-pretraining-with-swav?scriptVersionId=59516445)
- Want to read the SwAV paper, refer [this](https://arxiv.org/abs/2006.09882)

Hope you learnt something new in this notebook. Please consider **upvoting** if you found this notebook helpful in any way.